In [0]:
%pip install databricks-labs-dqx
dbutils.library.restartPython()

In [0]:
from pyspark.sql.functions import col
from databricks.labs.dqx.engine import DQEngine
from databricks.labs.dqx.rule import DQForEachColRule, DQDatasetRule
from databricks.labs.dqx import check_funcs
from databricks.labs.dqx.profiler.profiler import DQProfiler
from databricks.labs.dqx.profiler.generator import DQGenerator
from databricks.sdk import WorkspaceClient

In [0]:
df = spark.read.table("02_silver.staging.warcraftlogs_actors_players")

In [0]:
# Profile
ws = WorkspaceClient()
profiler = DQProfiler(ws)
summary_stats, profiles = profiler.profile(df)
checks = DQGenerator(ws).generate_dq_rules(profiles)

# Filter out is_in_range manually
checks = [
    c for c in checks
    if c.get("check", {}).get("function") != "is_in_range"
]

In [0]:
engine = DQEngine(spark)
valid_df, quarantine_df = engine.apply_checks_by_metadata_and_split(df, checks)

engine.save_results_in_table(
    output_df=valid_df,
    quarantine_df=quarantine_df,
    output_table="02_silver.warcraftlogs.actors_players",
    quarantine_table="02_silver.dq_monitoring.warcraftlogs_quarantine_actors_players"
)

In [0]:
spark.sql("DROP TABLE IF EXISTS 02_silver.staging.warcraftlogs_actors_players")